In [1]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)     

In [3]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True)

In [4]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    # print(inputs)
    # print(labels)
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
labels

tensor([8, 3, 1, 2, 4, 6, 7, 4, 3, 2, 7, 6, 3, 6, 0, 3, 2, 8, 9, 4, 6, 5, 4, 4,
        5, 7, 1, 2, 5, 8, 2, 4, 2, 4, 9, 2, 3, 1, 3, 4, 2, 7, 7, 4, 8, 5, 8, 0,
        1, 4, 5, 2, 5, 5, 5, 7, 1, 1, 8, 1, 3, 8, 2, 3])

In [6]:
len(train_loader)

938

In [7]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # nn.Dropout(p=0)就代表不用Dropout
        # 需要注意的是, Dropout也不是用了效果的一定好
        self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0.5), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(500,300), nn.Dropout(p=0.5), nn.Tanh())
        self.layer3 = nn.Sequential(nn.Linear(300,10), nn.Softmax(dim=1))
    
    def forward(self, x):
        # ([64, 1, 28, 28]) -> (64, 784)
        x = x.view(x.size()[0], -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [8]:
LR = 0.5
# 定义模型
model = Net()
# 定义损失函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.SGD(model.parameters(), LR)

In [9]:
def train():
    # 训练状态
    model.train()
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构 (64,10)
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.stop()
        
def test():
    # 测试状态
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(train_dataset)))

In [ ]:
for epoch in range(20):
    print('epoch:', epoch)
    train()
    test()